# Ray Tune Hyperparameter Search

https://docs.ray.io/en/latest/tune/index.html

In [ ]:
!pip install "ray[tune]" transformers datasets -q
!pip list | grep "transformers\|datasets\|torch\|ray" # show versions for reproducibility

     |████████████████████████████████| 54.5 MB 116 kB/s 
     |████████████████████████████████| 4.7 MB 53.5 MB/s 
     |████████████████████████████████| 365 kB 68.3 MB/s 
     |████████████████████████████████| 101 kB 12.4 MB/s 
     |████████████████████████████████| 6.6 MB 48.7 MB/s 
     |████████████████████████████████| 596 kB 73.3 MB/s 
     |████████████████████████████████| 141 kB 58.1 MB/s 
     |████████████████████████████████| 212 kB 74.0 MB/s 
     |████████████████████████████████| 127 kB 74.9 MB/s 
     |████████████████████████████████| 8.8 MB 60.9 MB/s 
     |████████████████████████████████| 4.1 MB 47.8 MB/s 
     |████████████████████████████████| 125 kB 75.0 MB/s 
     |████████████████████████████████| 466 kB 75.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 wh

In [ ]:
from datasets import load_dataset, load_metric
from transformers import (AutoModelForSequenceClassification, AutoTokenizer,
                          Trainer, TrainingArguments)

# I use a small model so it goes fast
model_ckpt = 'microsoft/xtremedistil-l6-h256-uncased'

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

# this is a small binary text classification dataset
dataset = load_dataset('glue', 'mrpc')
metric = load_metric('glue', 'mrpc')

# This will tokenize the examples
def encode(examples):
    outputs = tokenizer(
        examples['sentence1'], examples['sentence2'], truncation=True)
    return outputs

encoded_dataset = dataset.map(encode, batched=True)

# his function will be called at the beginning
# of each trial to create a new model
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(
        model_ckpt, return_dict=True)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Evaluate during training and a bit more often
# than the default to be able to prune bad trials early.
# Disabling tqdm is a matter of preference.
training_args = TrainingArguments(
    "ray-hp-search", 
    evaluation_strategy="steps", 
    eval_steps=500, 
    disable_tqdm=True
    )
trainer = Trainer(
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    model_init=model_init,
    compute_metrics=compute_metrics,
)

def raytune_objective(metrics):
    # if you don't pass this to the `hyperparameter_search` function, 
    # it will default to maximizing the sum of all metrics

    return metrics["eval_accuracy"]

def raytune_hp_space(trial):
    from ray import tune

    return {
        "learning_rate": tune.loguniform(1e-4, 1e-2),
        "num_train_epochs": tune.randint(lower=1, upper=6),
        "per_device_train_batch_size": tune.choice([4, 8, 16, 32, 64]),
        "weight_decay": tune.choice([0.0, 0.01, 0.05, 0.009]),
    }

# Default objective is the sum of all metrics
# when metrics are provided, so we have to maximize it.
best_trial = trainer.hyperparameter_search(
    direction="maximize", 
    backend="ray", 
    hp_space=raytune_hp_space,
    n_trials=10, # number of trials
    #compute_objective=raytune_objective,
    resources_per_trial={
            "cpu": 1,
            "gpu": 1, # can utilize multiple GPUs if available
        }
)
# additional kwargs will be passed to ray.tune.run (https://docs.ray.io/en/latest/tune/api_docs/execution.html)

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

...

== Status ==
Current time: 2022-07-28 22:09:50 (running for 00:06:58.62)
Memory usage on this node: 3.9/12.7 GiB
Using FIFO scheduling algorithm.
Resources requested: 1.0/2 CPUs, 1.0/1 GPUs, 0.0/7.34 GiB heap, 0.0/3.67 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /root/ray_results/_objective_2022-07-28_22-02-51
Number of trials: 10/10 (1 PENDING, 1 RUNNING, 8 TERMINATED)
+------------------------+------------+----------------+-----------------+--------------------+-------------------------------+----------------+-------------+
| Trial name             | status     | loc            |   learning_rate |   num_train_epochs |   per_device_train_batch_size |   weight_decay |   objective |
|------------------------+------------+----------------+-----------------+--------------------+-------------------------------+----------------+-------------|
| _objective_067ad_00008 | RUNNING    | 172.28.0.2:822 |     0.000292107 |                  3 |                            32 |        

(_objective pid=876) Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/xtremedistil-l6-h256-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
(_objective pid=876) You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
(_objective pid=876) /usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
(_objective pid=876)   FutureWarning,


== Status ==
Current time: 2022-07-28 22:10:35 (running for 00:07:43.10)
Memory usage on this node: 3.9/12.7 GiB
Using FIFO scheduling algorithm.
Resources requested: 1.0/2 CPUs, 1.0/1 GPUs, 0.0/7.34 GiB heap, 0.0/3.67 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /root/ray_results/_objective_2022-07-28_22-02-51
Number of trials: 10/10 (1 RUNNING, 9 TERMINATED)
+------------------------+------------+----------------+-----------------+--------------------+-------------------------------+----------------+-------------+
| Trial name             | status     | loc            |   learning_rate |   num_train_epochs |   per_device_train_batch_size |   weight_decay |   objective |
|------------------------+------------+----------------+-----------------+--------------------+-------------------------------+----------------+-------------|
| _objective_067ad_00009 | RUNNING    | 172.28.0.2:876 |     0.00925682  |                  1 |                            16 |          0     |  

2022-07-28 22:10:45,463	INFO tune.py:748 -- Total run time: 473.67 seconds (473.41 seconds for the tuning loop).


Result for _objective_067ad_00009:
  date: 2022-07-28_22-10-45
  done: true
  epoch: 1.0
  eval_accuracy: 0.6838235294117647
  eval_f1: 0.8122270742358079
  eval_loss: 0.6276770234107971
  eval_runtime: 0.3749
  eval_samples_per_second: 1088.374
  eval_steps_per_second: 136.047
  experiment_id: 3ab4b8d535c040619fe7bbd16ae02b25
  hostname: f2cea957d83c
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  objective: 1.4960506036475727
  pid: 876
  time_since_restore: 15.309271097183228
  time_this_iter_s: 15.309271097183228
  time_total_s: 15.309271097183228
  timestamp: 1659046245
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 067ad_00009
  warmup_time: 0.0037779808044433594
  
== Status ==
Current time: 2022-07-28 22:10:45 (running for 00:07:53.43)
Memory usage on this node: 3.7/12.7 GiB
Using FIFO scheduling algorithm.
Resources requested: 0/2 CPUs, 0/1 GPUs, 0.0/7.34 GiB heap, 0.0/3.67 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /root/ray_results/_ob

In [ ]:
best_trial

BestRun(run_id='067ad_00002', objective=1.8217900063251107, hyperparameters={'learning_rate': 0.000130667392380533, 'num_train_epochs': 5, 'per_device_train_batch_size': 32, 'weight_decay': 0.009})

# Ray with Hyperopt

http://hyperopt.github.io/hyperopt/

In [ ]:
!pip install -U hyperopt -q 
!pip list | grep "transformers\|datasets\|torch\|hyperopt"

     |████████████████████████████████| 1.6 MB 6.9 MB/s 
     |████████████████████████████████| 199 kB 46.8 MB/s 
datasets                      2.4.0
hyperopt                      0.2.7
tensorflow-datasets           4.0.1
torch                         1.12.0+cu113
torchaudio                    0.12.0+cu113
torchsummary                  1.5.1
torchtext                     0.13.0
torchvision                   0.13.0+cu113
transformers                  4.21.0
vega-datasets                 0.9.0


In [ ]:
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.schedulers import ASHAScheduler

from hyperopt import hp

hp_space = {
        "learning_rate": hp.loguniform("learning_rate", 1e-4, 1e-2),
        "num_train_epochs": hp.uniformint("num_train_epochs", lower=1, upper=6),
        "per_device_train_batch_size": hp.choice("per_device_train_batch_size", [4, 8, 16, 32, 64]),
        "weight_decay": hp.choice("weight_decay", [0.0, 0.01, 0.05, 0.009]),
    }

best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="ray",
    # hp_space=raytune_hp_space,
    # Choose among many libraries:
    # https://docs.ray.io/en/latest/tune/api_docs/suggestion.html
    search_alg=HyperOptSearch(metric="objective", mode="max", space=hp_space),
    # Choose among schedulers:
    # https://docs.ray.io/en/latest/tune/api_docs/schedulers.html
    scheduler=ASHAScheduler(metric="objective", mode="max", )
)


loading configuration file https://huggingface.co/microsoft/xtremedistil-l6-h256-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/e545113a9b5e643083a51ba645bed76c8df28ab46010b1ae39650ce7c668fe0c.142acdac8e7631616b334d09a95700d181a33a84fd7eb9ab46593634d0873dda
Model config BertConfig {
  "_name_or_path": "microsoft/xtremedistil-l6-h256-uncased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 8,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://h

== Status ==
Current time: 2022-07-27 19:32:39 (running for 00:00:00.28)
Memory usage on this node: 2.9/12.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 64.000: None | Iter 16.000: None | Iter 4.000: None | Iter 1.000: None
Resources requested: 1.0/2 CPUs, 1.0/1 GPUs, 0.0/7.36 GiB heap, 0.0/3.68 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /root/ray_results/_objective_2022-07-27_19-32-38
Number of trials: 1/20 (1 RUNNING)
+---------------------+----------+-----------------+-----------------+--------------------+-------------------------------+----------------+
| Trial name          | status   | loc             |   learning_rate |   num_train_epochs |   per_device_train_batch_size |   weight_decay |
|---------------------+----------+-----------------+-----------------+--------------------+-------------------------------+----------------|
| _objective_e02bb3a2 | RUNNING  | 172.28.0.2:1836 |       0.0030653 |                  3 |                             8 |    

(_objective pid=1836) Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/xtremedistil-l6-h256-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
(_objective pid=1836) You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
(_objective pid=1836) /usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
(_objective pid=1836)   FutureWarning,


== Status ==
Current time: 2022-07-27 19:32:49 (running for 00:00:10.32)
Memory usage on this node: 4.2/12.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 64.000: None | Iter 16.000: None | Iter 4.000: None | Iter 1.000: None
Resources requested: 1.0/2 CPUs, 1.0/1 GPUs, 0.0/7.36 GiB heap, 0.0/3.68 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /root/ray_results/_objective_2022-07-27_19-32-38
Number of trials: 2/20 (1 PENDING, 1 RUNNING)
+---------------------+----------+-----------------+-----------------+--------------------+-------------------------------+----------------+
| Trial name          | status   | loc             |   learning_rate |   num_train_epochs |   per_device_train_batch_size |   weight_decay |
|---------------------+----------+-----------------+-----------------+--------------------+-------------------------------+----------------|
| _objective_e02bb3a2 | RUNNING  | 172.28.0.2:1836 |     0.0030653   |                  3 |                         

(_objective pid=1907) Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/xtremedistil-l6-h256-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
(_objective pid=1907) You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
(_objective pid=1907) /usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
(_objective pid=1907)   FutureWarning,


...

== Status ==
Current time: 2022-07-27 19:48:38 (running for 00:15:59.35)
Memory usage on this node: 4.0/12.7 GiB
Using AsyncHyperBand: num_stopped=6
Bracket: Iter 64.000: None | Iter 16.000: None | Iter 4.000: 1.765661920159184 | Iter 1.000: 1.6431697972414994
Resources requested: 1.0/2 CPUs, 1.0/1 GPUs, 0.0/7.36 GiB heap, 0.0/3.68 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /root/ray_results/_objective_2022-07-27_19-32-38
Number of trials: 20/20 (1 PENDING, 1 RUNNING, 18 TERMINATED)
+---------------------+------------+-----------------+-----------------+--------------------+-------------------------------+----------------+-------------+
| Trial name          | status     | loc             |   learning_rate |   num_train_epochs |   per_device_train_batch_size |   weight_decay |   objective |
|---------------------+------------+-----------------+-----------------+--------------------+-------------------------------+----------------+-------------|
| _objective_049cdb06 | R

(_objective pid=2933) Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/xtremedistil-l6-h256-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
(_objective pid=2933) You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
(_objective pid=2933) /usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
(_objective pid=2933)   FutureWarning,


== Status ==
Current time: 2022-07-27 19:49:08 (running for 00:16:29.41)
Memory usage on this node: 4.0/12.7 GiB
Using AsyncHyperBand: num_stopped=7
Bracket: Iter 64.000: None | Iter 16.000: None | Iter 4.000: 1.765661920159184 | Iter 1.000: 1.5941300660435238
Resources requested: 1.0/2 CPUs, 1.0/1 GPUs, 0.0/7.36 GiB heap, 0.0/3.68 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /root/ray_results/_objective_2022-07-27_19-32-38
Number of trials: 20/20 (1 RUNNING, 19 TERMINATED)
+---------------------+------------+-----------------+-----------------+--------------------+-------------------------------+----------------+-------------+
| Trial name          | status     | loc             |   learning_rate |   num_train_epochs |   per_device_train_batch_size |   weight_decay |   objective |
|---------------------+------------+-----------------+-----------------+--------------------+-------------------------------+----------------+-------------|
| _objective_15e635ce | RUNNING    |

2022-07-27 19:49:27,733	INFO tune.py:748 -- Total run time: 1008.84 seconds (1008.38 seconds for the tuning loop).


Result for _objective_15e635ce:
  date: 2022-07-27_19-49-27
  done: true
  epoch: 1.09
  eval_accuracy: 0.6838235294117647
  eval_f1: 0.8122270742358079
  eval_loss: 0.626849889755249
  eval_runtime: 0.5448
  eval_samples_per_second: 748.883
  eval_steps_per_second: 93.61
  experiment_id: 08f80bd08feb4e958a148ec218993d97
  hostname: 2e9b942bd43a
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  objective: 1.4960506036475727
  pid: 2933
  time_since_restore: 23.955030918121338
  time_this_iter_s: 23.955030918121338
  time_total_s: 23.955030918121338
  timestamp: 1658951367
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 15e635ce
  warmup_time: 0.0037593841552734375
  
(_objective pid=2933) {'eval_loss': 0.626849889755249, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'eval_runtime': 0.5448, 'eval_samples_per_second': 748.883, 'eval_steps_per_second': 93.61, 'epoch': 1.09}
== Status ==
Current time: 2022-07-27 19:49:27 (running for 00:16:48.39)


In [ ]:
best_trial

BestRun(run_id='9e151236', objective=1.8322743731638997, hyperparameters={'learning_rate': 0.00010552453166133928, 'num_train_epochs': 4, 'per_device_train_batch_size': 32, 'weight_decay': 0.009})

# Optuna

You should restart the runtime after installing Optuna if you already ran the Ray hyperparameter search.

In [ ]:
!pip install optuna -q 
!pip list | grep "transformers\|datasets\|torch\|optuna"

     |████████████████████████████████| 308 kB 8.2 MB/s 
     |████████████████████████████████| 81 kB 7.6 MB/s 
     |████████████████████████████████| 209 kB 46.6 MB/s 
     |████████████████████████████████| 78 kB 6.6 MB/s 
     |████████████████████████████████| 112 kB 53.1 MB/s 
     |████████████████████████████████| 147 kB 55.9 MB/s 
     |████████████████████████████████| 49 kB 5.5 MB/s 
datasets                      2.4.0
optuna                        2.10.1
tensorflow-datasets           4.0.1
torch                         1.12.0+cu113
torchaudio                    0.12.0+cu113
torchsummary                  1.5.1
torchtext                     0.13.0
torchvision                   0.13.0+cu113
transformers                  4.21.0
vega-datasets                 0.9.0


In [ ]:
def optuna_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 7e-5, log=True),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 1, 5),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16, 32, 64]),
    }

def optuna_objective(metrics):
    # if you only want to optimize f1
    # pass to trainer.hyperparameter_search as compute_objective.
    # Default is to optimize sum of metrics (accuracy+f1 in this case)
    return metrics["eval_f1"]

# using same trainer as above
best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=optuna_hp_space,
    compute_objective=optuna_objective
)

[I 2022-07-27 20:09:08,277] A new study created in memory with name: no-name-1012c61e-f265-47cf-bab6-0aac1e8f9005
Trial:
loading configuration file https://huggingface.co/microsoft/xtremedistil-l6-h256-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/e545113a9b5e643083a51ba645bed76c8df28ab46010b1ae39650ce7c668fe0c.142acdac8e7631616b334d09a95700d181a33a84fd7eb9ab46593634d0873dda
Model config BertConfig {
  "_name_or_path": "microsoft/xtremedistil-l6-h256-uncased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 8,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers

{'train_runtime': 18.7338, 'train_samples_per_second': 195.796, 'train_steps_per_second': 24.501, 'train_loss': 0.5082604568248741, 'epoch': 1.0}


[I 2022-07-27 20:09:28,790] Trial 0 finished with value: 0.883248730964467 and parameters: {'learning_rate': 3.22581283235675e-05, 'num_train_epochs': 1, 'per_device_train_batch_size': 16}. Best is trial 0 with value: 0.883248730964467.
Trial:


{'eval_loss': 0.43409672379493713, 'eval_accuracy': 0.8308823529411765, 'eval_f1': 0.883248730964467, 'eval_runtime': 0.5985, 'eval_samples_per_second': 681.755, 'eval_steps_per_second': 85.219, 'epoch': 1.0}


loading configuration file https://huggingface.co/microsoft/xtremedistil-l6-h256-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/e545113a9b5e643083a51ba645bed76c8df28ab46010b1ae39650ce7c668fe0c.142acdac8e7631616b334d09a95700d181a33a84fd7eb9ab46593634d0873dda
Model config BertConfig {
  "_name_or_path": "microsoft/xtremedistil-l6-h256-uncased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 8,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://h

...

{'eval_loss': 0.30488160252571106, 'eval_accuracy': 0.8921568627450981, 'eval_f1': 0.9219858156028369, 'eval_runtime': 0.5796, 'eval_samples_per_second': 703.929, 'eval_steps_per_second': 87.991, 'epoch': 2.18}


loading configuration file https://huggingface.co/microsoft/xtremedistil-l6-h256-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/e545113a9b5e643083a51ba645bed76c8df28ab46010b1ae39650ce7c668fe0c.142acdac8e7631616b334d09a95700d181a33a84fd7eb9ab46593634d0873dda
Model config BertConfig {
  "_name_or_path": "microsoft/xtremedistil-l6-h256-uncased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 8,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://h

{'loss': 0.5349, 'learning_rate': 7.576941113638977e-06, 'epoch': 1.09}


[I 2022-07-27 20:22:44,351] Trial 19 pruned. 


{'eval_loss': 0.4520810544490814, 'eval_accuracy': 0.8284313725490197, 'eval_f1': 0.8813559322033899, 'eval_runtime': 0.5895, 'eval_samples_per_second': 692.087, 'eval_steps_per_second': 86.511, 'epoch': 1.09}


In [ ]:
print(best_trial)

BestRun(run_id='9', objective=0.9249563699825479, hyperparameters={'learning_rate': 3.9531397929381135e-05, 'num_train_epochs': 3, 'per_device_train_batch_size': 32})
